In [1]:
import comet_ml
from comet_ml.integration.pytorch import log_model

import torch
from torchvision.datasets import ImageFolder
from torch.utils.data.dataloader import DataLoader
import torchvision.transforms as vt


#import os
import yaml
from munch import DefaultMunch
from tqdm.notebook import trange,tqdm
#from tqdm.notebook import tqdm
import random
import numpy as np
from wgan_gp import WGAN_GP

#torch.set_float32_matmul_precision('medium')
def set_seed(seed):
        torch.backends.cudnn.deterministic = True
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)


cfg_path = "config.yml"
with open(cfg_path, "r") as f:
    print(f"Loading config file: {cfg_path}")
    cfg = yaml.safe_load(f)
cfg = DefaultMunch.fromDict(cfg)
print(cfg)
set_seed(cfg.seed)

transforms = vt.Compose([vt.ToTensor(),vt.Normalize(0.5, 0.5),
    vt.Resize((cfg.imsize, cfg.imsize),antialias=True)])

dataset = ImageFolder(
    root=cfg.data_dir, transform=transforms
)
    
dataloader = DataLoader(
    dataset,
    batch_size=cfg.batch_size,
    shuffle=True,
    num_workers=4,
    drop_last=True,
)
model=WGAN_GP(cfg)
start_epoch=model.starting_epoch
loop = trange(start_epoch,start_epoch+cfg.epochs, desc="Epoch: ",leave=True)
#loop = tqdm(start_epoch,start_epoch+cfg.epochs, desc="Epoch: ", ncols=75,leave=True)



experiment = comet_ml.Experiment(project_name=cfg.comet_project, log_graph=True,workspace=cfg.comet_workspace)
                #                         auto_metric_logging=False, #auto_output_logging=False)
experiment.set_name(cfg.precision if cfg.use_fabric else "full" )
experiment.log_parameters(cfg)
experiment.log_parameter("starting epoch",start_epoch)
## log the model graph
mstr=str(model.generator)+"\n"+str(model.discrim)
experiment.set_model_graph(mstr)

for epoch in loop:
    loss_d,loss_g=model.train_epoch(dataloader)
    loop.set_postfix(loss_d=loss_d,loss_g=loss_g)
    metrics={'loss_d':loss_d,'loss_g':loss_g}
    experiment.log_metrics(metrics, epoch=epoch)
    if (epoch+1) % cfg.save_model_freq == 0:
        #print(f"Saving model at epoch {epoch}")
        model.save_model(epoch)
        
        #fid=model.compute_fid(dataloader=dataloader)
        #experiment.log_metric("fid",fid,epoch=epoch)
    if(epoch+1) % cfg.save_image_freq == 0:
        img=model.save_images(epoch,32)
        experiment.log_image(img)
        print(f"Saving images at epoch {epoch}")

model_checkpoint = {
        "epoch": epoch,
        "g_state_dict": model.generator.state_dict(),
        "d_state_dict": model.discrim.state_dict(),
        "optG_state_dict": model.optG.state_dict(),
        "optD_state_dict": model.optD.state_dict(),
        "loss_d":loss_d,
        "loss_g":loss_g}

log_model(experiment, model_checkpoint, model_name="GAN-WP")
experiment.end()

Loading config file: config.yml
DefaultMunch(None, {'name': 'WGAN-GP', 'seed': 42, 'device': 'cuda', 'data_dir': '/home/user/PyTorch-Lightning-GAN/GAN/celeba', 'zdim': 128, 'imsize': 64, 'img_ch': 3, 'w_gp': 10, 'd_iter_per_g': 5, 'lr': DefaultMunch(None, {'g': 0.0001, 'd': 0.0001}), 'batch_size': 256, 'num_workers': 4, 'norm_type': DefaultMunch(None, {'g': 'GroupNorm', 'd': 'GroupNorm'}), 'final_activation': DefaultMunch(None, {'g': 'tanh'}), 'use_fabric': True, 'precision': 'bf16-mixed', 'resume': False, 'epochs': 50, 'save_image_freq': 10, 'save_model_freq': 100, 'weights_dir': 'chkpt', 'images_dir': 'images', 'comet_workspace': 'wgan', 'comet_project': 'wgan-gp', 'samples_dir': 'samples', 'num_sample_epochs': 20})


Using bfloat16 Automatic Mixed Precision (AMP)
You are using a CUDA device ('NVIDIA GeForce RTX 3070') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


Epoch:   0%|                                        | 0/50 [00:00<?, ?it/s]

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/wgan/wgan-gp/3515b725ded241288e5cbc84798de97c



Iteration:   0%|          | 0/791 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/791 [00:00<?, ?it/s]

Iteration:   0%|          | 0/791 [00:00<?, ?it/s]

Iteration:   0%|          | 0/791 [00:00<?, ?it/s]

Iteration:   0%|          | 0/791 [00:00<?, ?it/s]

Iteration:   0%|          | 0/791 [00:00<?, ?it/s]